This project implements a Python-based Ticketing System designed to manage ticket creation, tracking, and processing using structured data handling and object-oriented programming principles. The system models real-world ticket workflows by maintaining organized ticket records, enforcing controlled updates, and supporting efficient access to ticket information.

The design emphasizes clarity, scalability, and data integrity, making it suitable for extension into larger systems such as customer support platforms, event booking systems, or service request management tools.

The code below will be the official code with a seating capacity of 500.

In [ ]:
# Import libraries
import heapq
import random

# Create a class for the ticket
class Ticket:
  def __init__(self, ticket_id, name, number, email, priority, seat=None):
    self.ticket_id = ticket_id
    self.name = name
    self.number = number
    self.email = email
    self.priority = priority
    self.seat = seat

  # Create a function to print the object info (ie. ticket info)
  def __repr__(self):
    return (f"\nTicketID : {self.ticket_id}\n"
              f"Name : {self.name}\n"
              f"Phone Number : {self.number}\n"
              f"Email : {self.email}\n"
              f"Seat : {self.seat}\n"
              f"Priority : {self.priority}\n")

# Create class for seating managment
class Seating:

# Initialize the seating
  def __init__(self, rows=20, cols=25):
    self.rows = rows
    self.cols = cols

    # Initalize structure for seating-dynamic array
    # 2D list
    self.seats = [[None for _ in range(cols)] for _ in range(rows)]

    # Initialize the dictionary for the customer details
    self.purchased_tickets = {}

    # Initialize the priority queue
    self.queue = []
    # Job count initialization for queue to keep track of equal priorities
    self.job_count = 0


  def display_seats(self):

    # Add the column numbers
    # Print a space on the left side
    print("   ",end=" ")
    for i in range(self.cols):
      print(f"{i:2d} ", end="  ")
    # Print new line
    print()
    # Print ----
    print("  " + "-----" * self.cols)


    for i in range(self.rows):
      # Print | before each row
      print(f"{i} |",end=" ")

      for j in range(self.cols):
        if self.seats[i][j] is None:
          # Print O for empty seats
          print("O  ", end = "  ")
        else:
          # Print X for taken seats
          print("X  ", end = "  ")
      # New line after each row
      print("\n")


  def buy_ticket(self):

    # Display the grid to customer
    self.display_seats()

    # Create ticket ID
    ticket_set = set(self.purchased_tickets.keys())
    # Check which id's exist as to not create duplicates
    while True:
      # Can change this if the number ever increases drastically
      ticket_id = random.randint(1000,4000)
      if ticket_id not in ticket_set:
        break

    # Get customer dets
    name = input("Please enter your name: ")
    number = input("Please enter your phone number: ")
    email = input("Please enter your email: ")
    # Priority of 1 automatically assigned to people buying seats when tickets are available-customer will get choice afterwards because they can for example "pay" more money to be of hgher priority
    priority = 1

    # Check if there are "empty" seats before asking customer-if there are then continue with code below-else break out of loop and place into queue
    empty_seats = False
    for row in self.seats:
      for seat in row:
        if seat is None:
          empty_seats = True
          break
    if empty_seats:
      # Ask customer to select seating by selecting row and col value
      print("Please select the row and column in which you choose to sit within the grid. Seats that are empty are represented by 'O'.")
      try:
        row = int(input("Select row: "))
        col = int(input("Select column: "))
        seat = (row, col)

        # Add some sort of error handling here for if the value is beyond the grid
        if row < 0 or row >= self.rows or col < 0 or col >= self.cols:
          return "The seating is out of bounds, please select seat within the grid."

      except ValueError:
        return "Please select a valid value."

      ticket = Ticket(ticket_id, name, number, email, priority, seat)

      # Add ticket to the tickets sold dictionary
      self.purchased_tickets[ticket_id] = ticket

      # Update the seat
      self.seats[row][col] = ticket

      return f"Ticket booked with priority 1 and TicketID {ticket_id}."

    else:
      # There are no more seats left
      print("There are no more empty seats in the venue. We will be placing you into a waiting list.\n"
            "You can choose between priorities with each priority ticket varying in price.\n"
            "The priority will determine where you stand in queue.")

      # Define the chosen priority value
      priority = int(input("Choose your priority from 1, 2, or 3: "))
      ticket = Ticket(ticket_id, name, number, email, priority, None)


      # Push into the heapq
      heapq.heappush(self.queue, (priority, self.job_count, ticket))
      self.job_count += 1
      return f"Ticket with priority {priority} and TicketID {ticket_id} added to waitlist. Note you will be assigned seating once a seat opens up."



  def cancel_ticket(self, ticket_id):

    try:
      # If ticket is in purchased_tickets
      if ticket_id in self.purchased_tickets:
        # Define the ticket variable for ease of access
        cancelled_ticket = self.purchased_tickets[ticket_id]
        # Obtain the seat values
        row, col = cancelled_ticket.seat

        # Emtpy the seat
        self.seats[row][col] = None

        # Delete the ticket_id
        del self.purchased_tickets[ticket_id]
        print("Ticket has been cancelled.\n")

        # Handle priority queue
        if self.queue:
          priority, order, new_ticket = heapq.heappop(self.queue)
          # Reassign the seating
          new_ticket.seat = (row, col)
          # Add the ticket into the sold tickets dict
          self.purchased_tickets[new_ticket.ticket_id] = new_ticket
          # Update the grid
          self.seats[row][col] = new_ticket
          return f"Ticket queue has been adjusted. Ticket ID {new_ticket.ticket_id} is now in seat {(row,col)}\n"
        # If queue is empty do nothing
        else:
          return "No adjustments made to the queue."


      # Check if ticket in queue
      # Create new queue as e are attempting to access element at arbritray position and will need to adjust queue
      new_queue = []
      found = False
      for ticket in self.queue:
        # Access the second elemenet of ticket (ie. ticket# from priority list)
        if ticket[2].ticket_id == ticket_id:
          found = True
        else:
          new_queue.append(ticket)
      if found:
        self.queue = new_queue
        heapq.heapify(self.queue)
        print(f"Ticket with ID {ticket_id} removed from queue.")
      else:
        return f"Ticket with ID {ticket_id} not found in our records."
      return f"Ticket with ID {ticket_id} has been deleted."
    except ValueError:
      return f"Ticket with ID {ticket_id} not found in our records."



  def modifications(self, ticket_id, new_priority=None, new_row=None, new_col=None):
    try:
      # If updating seating-check if new values are in bounds
      if new_col is not None or new_row is not None:
        if new_col < 0 or new_col >= self.cols or new_row < 0 or new_row >= self.rows:
          return "The new seating you have selected is out of bounds."

        # Check if there is already someone in the seat
        if self.seats[new_row][new_col] is not None:
          return "This seat is already taken. Please try to select another seat or keep your original."

      found_purchased = False
      # Modify tickets that have seats
      if ticket_id in self.purchased_tickets:
        found_purchased = True
        # Define the ticket to modify
        ticket = self.purchased_tickets[ticket_id]

        # Changing priority
        if new_priority is not None:
          ticket.priority = new_priority
          return f"Ticket priority changed to {new_priority}"

        if new_col is not None or new_row is not None:
          # Remove person from their original seat
          old_row, old_col = ticket.seat
          self.seats[old_row][old_col] = None
          # Add person to new seat
          ticket.seat = (new_row,new_col)
          self.seats[new_row][new_col] = ticket
          return f"Seat changed from {old_row,old_col} to {(new_row,new_col)}."


      # Modify the priority for customers in queue-they will not have seats
      new_queue = []
      found_queue = False
      for ticket in self.queue:
        if new_priority is not None and ticket[2].ticket_id == ticket_id:
          found_queue = True
          ticket[2].priority = new_priority
          updated_ticket = (new_priority, ticket[1], ticket[2])
          new_queue.append(updated_ticket)
        else:
          # Append the non matching tickets to a new queue
          new_queue.append(ticket)
      # If a match is found in the queue reorganize the queue by priority
      if found_queue:
        self.queue = new_queue
        heapq.heapify(self.queue)
        return f"Ticket with ID {ticket_id} assigned new priority of {new_priority}."

      # Case where ticket is not found in either purchased tickets or the queue
      if not found_queue and not found_purchased:
        return f"Ticket with ID {ticket_id} is not in the waitlist nor our records of sold tickets."

    except ValueError:
      return f"Ticket with ID {ticket_id} not found."



  def lookup(self,ticket_id):
    # Check where the ticket is
    if ticket_id in self.purchased_tickets:
      # Return all info regarding ticket
      return self.purchased_tickets[ticket_id]

    # Check if ticket exists in queue
    for ticket in self.queue:
      if ticket[2].ticket_id == ticket_id:
        return ticket[2]

    return "Ticket not found"



  def expand_seating(self,new_capacity):
    current_capacity = self.rows * self.cols
    old_rows = self.rows
    # Error handling
    if new_capacity <= current_capacity:
      return "New capacity must be greater than current of {current_capacity}."
    # Define how many new seats we require
    additional_seats = new_capacity - current_capacity
    # Change the amount of rows we have
    extra_rows = additional_seats // self.cols
    # Update seat rows
    for _ in range(extra_rows):
      self.seats.append([None for _ in range(self.cols)])
    # Update rows
    self.rows = self.rows + extra_rows

    # Add a counter variable to see how many people popped from queue
    popped = 0
    # Manage queue
    for  i in range(old_rows, self.rows):
      for j in range(self.cols):
        if self.seats[i][j] is None and self.queue:
          # Pop the next item from queue
          priority, order, waitlist_tic = heapq.heappop(self.queue)
          # Assign the next available seat to the poped item
          waitlist_tic.seat = (i,j)
          # Update our dict of purchased tickets
          self.purchased_tickets[waitlist_tic.ticket_id] = waitlist_tic
          # Assign the ticket to our grid
          self.seats[i][j] = waitlist_tic
          print( f"Ticket queue has been adjusted. Ticket with ID {waitlist_tic.ticket_id} assiged to seat {waitlist_tic.seat}.\n")
          popped += 1
        if not self.queue:
          break
      if not self.queue:
        break

    # Display seating with tickets added from queue
    self.display_seats()
    return f"{extra_rows} extra rows, and {additional_seats} additional seats added to create a capactiy of {new_capacity} in the venue.\n{popped} people in queue have now been seated."



  def main(self):
    # For modifications check if the ticket info matches to itself first (ie. if the customer is trying to update value to the same value)
    while True:
      try:
        print(
         "Hello, welcome to the ticketing system for our concert.\n"
         "We have the following options:\n"
            "1) Book a ticket\n"
            "2) Cancel a ticket\n"
            "3) Modify an existing ticket\n"
            "4) Lookup your ticket info\n"
            "5) Expand the seating capacity\n")

        choice = int(input("Please select an option: "))
      except ValueError:
        print("Please select one valid option.")
        continue

      # First Choice-book
      if choice == 1:
        result = self.buy_ticket()
        if result is not None:
          print(result)
          break

      # Second choice-cancel
      elif choice == 2:
        ticket_id = int(input("Please enter your TicketID: "))
        result = self.cancel_ticket(ticket_id)
        if result is not None:
          print(result)
          break

      # Third Choice-modify
      elif choice == 3:
        ticket_id = int(input("Please enter your TicketID: "))
        try:
          adjust = int(input(
                    "\nWhat would you like to modify? Please select a number from below:\n"
                          "1) Priority\n"
                          "2) Seat\n"))
        # Error handling if customer inputs two moddifications-we are handling one at a time
        except ValueError:
          return "\nPlease select 1 option.\n"
          continue

        if adjust ==1:
          new_priority = int(input("Please enter your new priority: "))
          result = self.modifications(ticket_id, new_priority, None, None)
        elif adjust == 2:
          self.display_seats()
          new_row = int(input("Please enter your new row value: "))
          new_col = int(input("Please enter your new column value: "))
          result = self.modifications(ticket_id, None, new_row, new_col)
        else:
          print("Please select a valid option\n")
          continue
        if result is not None:
          print(result)
          break

      # Fourth Choice-lookup
      elif choice == 4:
        ticket_id = int(input("Please enter your TicketID: "))
        result = self.lookup(ticket_id)
        if result is not None:
          print(result)
          break

      # Fifth Choice-change capacity
      elif choice == 5:
        new_capacity = int(input("Please enter the new capacity: "))
        result = self.expand_seating(new_capacity)
        if result is not None:
          print(result)
          break
      else:
        #return "Please select a valid option."
        print("\nPlease select a valid option\n")
        continue


# create ticket booker manager
ticket_booker = Seating()








Below are some test cases

In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 1
     0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   
  -----------------------------------------------------------------------------------------------------------------------------
0 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    

1 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    

2 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    

3 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O 

In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 4
Please enter your TicketID: 1590

TicketID : 1590
Name : Harlene
Phone Number : 245454
Email : h@hot.ca
Seat : (3, 6)
Priority : 1



In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 3
Please enter your TicketID: 1590

What would you like to modify? Please select a number from below:
1) Priority
2) Seat
1
Please enter your new priority: 3
Ticket priority changed to 3


In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 3
Please enter your TicketID: 1590

What would you like to modify? Please select a number from below:
1) Priority
2) Seat
2
     0    1    2    3    4    5    6    7    8    9   10   11   12   13   14   15   16   17   18   19   20   21   22   23   24   
  -----------------------------------------------------------------------------------------------------------------------------
0 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    

1 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    

2 | O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O    O

In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 2
Please enter your TicketID: 1590
Ticket has been cancelled.

No adjustments made to the queue.


The code below acts as a test case to display what happens when we are changing the capactiy (ie. dealing with the queue). It is the same code as above with the seating changed to initialize at 1

In [ ]:
# Import libraries
import heapq
import random

# Create a class for the ticket
class Ticket:
  def __init__(self, ticket_id, name, number, email, priority, seat=None):
    self.ticket_id = ticket_id
    self.name = name
    self.number = number
    self.email = email
    self.priority = priority
    self.seat = seat

  # Create a function to print the object info (ie. ticket info)
  def __repr__(self):
    return (f"\nTicketID : {self.ticket_id}\n"
              f"Name : {self.name}\n"
              f"Phone Number : {self.number}\n"
              f"Email : {self.email}\n"
              f"Seat : {self.seat}\n"
              f"Priority : {self.priority}\n")



# Create class for seating managment
class Seating:

# Initialize the seating
  def __init__(self, rows=1, cols=1):
    self.rows = rows
    self.cols = cols

    # Initalize structure for seating-dynamic array
    # 2D list
    self.seats = [[None for _ in range(cols)] for _ in range(rows)]

    # Initialize the dictionary for the customer details
    self.purchased_tickets = {}

    # Initialize the priority queue
    self.queue = []
    # Job count initialization for queue to keep track of equal priorities
    self.job_count = 0


  def display_seats(self):

    # Add the column numbers
    # Print a space on the left side
    print("   ",end=" ")
    for i in range(self.cols):
      print(f"{i:2d} ", end="  ")
    # Print new line
    print()
    # Print ----
    print("  " + "-----" * self.cols)


    for i in range(self.rows):
      # Print | before each row
      print(f"{i} |",end=" ")

      for j in range(self.cols):
        if self.seats[i][j] is None:
          # Print O for empty seats
          print("O  ", end = "  ")
        else:
          # Print X for taken seats
          print("X  ", end = "  ")
      # New line after each row
      print("\n")


  def buy_ticket(self):

    # Display the grid to customer
    self.display_seats()

    # Create ticket ID
    ticket_set = set(self.purchased_tickets.keys())
    # Check which id's exist as to not create duplicates
    while True:
      # Can change this if the number ever increases drastically
      ticket_id = random.randint(1000,4000)
      if ticket_id not in ticket_set:
        break

    # Get customer dets
    name = input("Please enter your name: ")
    number = input("Please enter your phone number: ")
    email = input("Please enter your email: ")
    # Priority of 1 automatically assigned to people buying seats when tickets are available-customer will get choice afterwards because they can for example "pay" more money to be of hgher priority
    priority = 1

    # Check if there are "empty" seats before asking customer-if there are then continue with code below-else break out of loop and place into queue
    empty_seats = False
    for row in self.seats:
      for seat in row:
        if seat is None:
          empty_seats = True
          break
    if empty_seats:
      # Ask customer to select seating by selecting row and col value
      print("Please select the row and column in which you choose to sit within the grid. Seats that are empty are represented by 'O'.")
      try:
        row = int(input("Select row: "))
        col = int(input("Select column: "))
        seat = (row, col)

        # Add some sort of error handling here for if the value is beyond the grid
        if row < 0 or row >= self.rows or col < 0 or col >= self.cols:
          return "The seating is out of bounds, please select seat within the grid."

      except ValueError:
        return "Please select a valid value."

      ticket = Ticket(ticket_id, name, number, email, priority, seat)

      # Add ticket to the tickets sold dictionary
      self.purchased_tickets[ticket_id] = ticket

      # Update the seat
      self.seats[row][col] = ticket

      return f"Ticket booked with priority 1 and TicketID {ticket_id}."

    else:
      # There are no more seats left
      print("There are no more empty seats in the venue. We will be placing you into a waiting list.\n"
            "You can choose between priorities with each priority ticket varying in price.\n"
            "The priority will determine where you stand in queue.")

      # Define the chosen priority value
      priority = int(input("Choose your priority from 1, 2, or 3: "))
      ticket = Ticket(ticket_id, name, number, email, priority, None)


      # Push into the heapq
      heapq.heappush(self.queue, (priority, self.job_count, ticket))
      self.job_count += 1
      return f"Ticket with priority {priority} and TicketID {ticket_id} added to waitlist."



  def cancel_ticket(self, ticket_id):

    try:
      # If ticket is in purchased_tickets
      if ticket_id in self.purchased_tickets:
        # Define the ticket variable for ease of access
        cancelled_ticket = self.purchased_tickets[ticket_id]
        # Obtain the seat values
        row, col = cancelled_ticket.seat

        # Emtpy the seat
        self.seats[row][col] = None

        # Delete the ticket_id
        del self.purchased_tickets[ticket_id]
        print("Ticket has been cancelled.\n")

        # Handle priority queue
        if self.queue:
          priority, order, new_ticket = heapq.heappop(self.queue)
          # Reassign the seating
          new_ticket.seat = (row, col)
          # Add the ticket into the sold tickets dict
          self.purchased_tickets[new_ticket.ticket_id] = new_ticket
          # Update the grid
          self.seats[row][col] = new_ticket
          return f"Ticket queue has been adjusted. Ticket ID {new_ticket.ticket_id} is now in seat {(row,col)}\n"
        # If queue is empty do nothing
        else:
          return "No adjustments made to the queue."


      # Check if ticket in queue
      new_queue = []
      found = False
      for ticket in self.queue:
        # Access the second elemenet of ticket (ie. ticket# from priority list)
        if ticket[2].ticket_id == ticket_id:
          found = True
        else:
          new_queue.append(ticket)
      if found:
        self.queue = new_queue
        heapq.heapify(self.queue)
        print(f"Ticket with ID {ticket_id} removed from queue.")
      else:
        return f"Ticket with ID {ticket_id} not found in our records."
      return f"Ticket with ID {ticket_id} has been deleted."
    except ValueError:
      return f"Ticket with ID {ticket_id} not found in our records."




  def modifications(self, ticket_id, new_priority=None, new_row=None, new_col=None):
    try:
      # If updating seating-check if new values are in bounds
      if new_col is not None or new_row is not None:
        if new_col < 0 or new_col >= self.cols or new_row < 0 or new_row >= self.rows:
          return "The new seating you have selected is out of bounds."

        # Check if there is already someone in the seat
        if self.seats[new_row][new_col] is not None:
          return "This seat is already taken. Please try to select another seat or keep your original."

      found_purchased = False
      # Modify tickets that have seats
      if ticket_id in self.purchased_tickets:
        found_purchased = True
        # Define the ticket to modify
        ticket = self.purchased_tickets[ticket_id]

        # Changing priority
        if new_priority is not None:
          ticket.priority = new_priority
          return f"Ticket priority changed to {new_priority}"

        if new_col is not None or new_row is not None:
          # Remove person from their original seat
          old_row, old_col = ticket.seat
          self.seats[old_row][old_col] = None
          # Add person to new seat
          ticket.seat = (new_row,new_col)
          self.seats[new_row][new_col] = ticket
          return f"Seat changed from {old_row,old_col} to {(new_row,new_col)}."


      # Modify the priority for customers in queue-they will not have seats
      new_queue = []
      found_queue = False
      for ticket in self.queue:
        if new_priority is not None and ticket[2].ticket_id == ticket_id:
          found_queue = True
          ticket[2].priority = new_priority
          updated_ticket = (new_priority, ticket[1], ticket[2])
          new_queue.append(updated_ticket)
        else:
          # Append the non matching tickets to a new queue
          new_queue.append(ticket)
      # If a match is found in the queue reorganize the queue by priority
      if found_queue:
        self.queue = new_queue
        heapq.heapify(self.queue)
        return f"Ticket with TicketID {ticket_id} assigned new priority of {new_priority}."

      # Case where ticket is not found in either purchased tickets or the queue
      if not found_queue and not found_purchased:
        return f"Ticket with ID {ticket_id} is not in the waitlist nor our records of sold tickets."

    except ValueError:
      return f"Ticket with ID {ticket_id} not found."



  def lookup(self,ticket_id):
    # Check where the ticket is
    if ticket_id in self.purchased_tickets:
      # Return all info regarding ticket
      return self.purchased_tickets[ticket_id]

    # Check if ticket exists in queue
    for ticket in self.queue:
      if ticket[2].ticket_id == ticket_id:
        return ticket[2]

    return "Ticket not found"



  def expand_seating(self,new_capacity):
    current_capacity = self.rows * self.cols
    old_rows = self.rows
    # Error handling
    if new_capacity <= current_capacity:
      return "New capacity must be greater than current of {current_capacity}."
    # Define how many new seats we require
    additional_seats = new_capacity - current_capacity
    # Change the amount of rows we have
    extra_rows = additional_seats // self.cols
    # Update seat rows
    for _ in range(extra_rows):
      self.seats.append([None for _ in range(self.cols)])
    # Update rows
    self.rows = self.rows + extra_rows

    # Add a counter variable to see how many people popped from queue
    popped = 0
    # Manage queue
    for  i in range(old_rows, self.rows):
      for j in range(self.cols):
        if self.seats[i][j] is None and self.queue:
          # Pop the next item from queue
          priority, order, waitlist_tic = heapq.heappop(self.queue)
          # Assign the next available seat to the poped item
          waitlist_tic.seat = (i,j)
          # Update our dict of purchased tickets
          self.purchased_tickets[waitlist_tic.ticket_id] = waitlist_tic
          # Assign the ticket to our grid
          self.seats[i][j] = waitlist_tic
          print( f"Ticket queue has been adjusted. Ticket with ID {waitlist_tic.ticket_id} assiged to seat {waitlist_tic.seat}.\n")
          popped += 1
        if not self.queue:
          break
      if not self.queue:
        break

    # Display seating with tickets added from queue
    self.display_seats()
    return f"{extra_rows} extra rows, and {additional_seats} additional seats added to create a capactiy of {new_capacity} in the venue.\n{popped} people in queue have now been seated."



  def main(self):
    # For modifications check if the ticket info matches to itself first (ie. if the customer is trying to update value to the same value)
    while True:

      try:
        print(
         "Hello, welcome to the ticketing system for our concert.\n"
         "We have the following options:\n"
            "1) Book a ticket\n"
            "2) Cancel a ticket\n"
            "3) Modify an existing ticket\n"
            "4) Lookup your ticket info\n"
            "5) Expand the seating capacity\n")

        choice = int(input("Please select an option: "))
      except ValueError:
        print("Please select one valid option.")
        continue

      # First Choice-book
      if choice == 1:
        result = self.buy_ticket()
        if result is not None:
          print(result)
          break

      # Second choice-cancel
      elif choice == 2:
        ticket_id = int(input("Please enter your TicketID: "))
        result = self.cancel_ticket(ticket_id)
        if result is not None:
          print(result)
          break

      # Third Choice-modify
      elif choice == 3:
        ticket_id = int(input("Please enter your TicketID: "))
        try:
          adjust = int(input(
                    "\nWhat would you like to modify? Please select a number from below:\n"
                          "1) Priority\n"
                          "2) Seat\n"))
        # Error handling if customer inputs two moddifications-we are handling one at a time
        except ValueError:
          return "\nPlease select 1 option.\n"
          continue

        if adjust ==1:
          new_priority = int(input("Please enter your new priority: "))
          result = self.modifications(ticket_id, new_priority, None, None)
        elif adjust == 2:
          self.display_seats()
          new_row = int(input("Please enter your new row value: "))
          new_col = int(input("Please enter your new column value: "))
          result = self.modifications(ticket_id, None, new_row, new_col)
        else:
          print("Please select a valid option\n")
          continue
        if result is not None:
          print(result)
          break

      # Fourth Choice-lookup
      elif choice == 4:
        ticket_id = int(input("Please enter your TicketID: "))
        result = self.lookup(ticket_id)
        if result is not None:
          print(result)
          break

      # Fifth Choice-change capacity
      elif choice == 5:
        new_capacity = int(input("Please enter the new capacity: "))
        result = self.expand_seating(new_capacity)
        if result is not None:
          print(result)
          break
      else:
        #return "Please select a valid option."
        print("\nPlease select a valid option\n")
        continue



# create ticket booker manager
ticket_booker = Seating()








Below are some test cases to show the handling of the queue with the expansion fo the venue seating.

In [ ]:

# Call ticket_booker.main() each time we want to use code
ticket_booker.main()


Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 1
     0   
  -----
0 | O    

Please enter your name: Harlene
Please enter your phone number: 5673335555
Please enter your email: h@go.ca
Please select the row and column in which you choose to sit within the grid. Seats that are empty are represented by 'O'.
Select row: 0
Select column: 0
Ticket booked with priority 1 and TicketID 2206.


In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 1
     0   
  -----
0 | X    

Please enter your name: Bob
Please enter your phone number: 2423452
Please enter your email: b@go.ca
There are no more empty seats in the venue. We will be placing you into a waiting list.
You can choose between priorities with each priority ticket varying in price.
The priority will determine where you stand in queue.
Choose your priority from 1, 2, or 3: 2
Ticket with priority 2 and TicketID 3786 added to waitlist.


In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 3
Please enter your TicketID: 3786 

What would you like to modify? Please select a number from below:
1) Priority
2) Seat
1
Please enter your new priority: 1
Ticket with TicketID 3786 assigned new priority of 1.


In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 1
     0   
  -----
0 | X    

Please enter your name: Tom
Please enter your phone number: 13423535
Please enter your email: t@hot.ca
There are no more empty seats in the venue. We will be placing you into a waiting list.
You can choose between priorities with each priority ticket varying in price.
The priority will determine where you stand in queue.
Choose your priority from 1, 2, or 3: 3
Ticket with priority 3 and TicketID 1473 added to waitlist.


In [ ]:
ticket_booker.main()

Hello, welcome to the ticketing system for our concert.
We have the following options:
1) Book a ticket
2) Cancel a ticket
3) Modify an existing ticket
4) Lookup your ticket info
5) Expand the seating capacity

Please select an option: 5
Please enter the new capacity: 4
Ticket queue has been adjusted. Ticket with ID 3786 assiged to seat (1, 0).

Ticket queue has been adjusted. Ticket with ID 1473 assiged to seat (2, 0).

     0   
  -----
0 | X    

1 | X    

2 | X    

3 | O    

3 extra rows, and 3 additional seats added to create a capactiy of 4 in the venue.
2 people in queue have now been seated.


Below is the AI generated code

In [ ]:
import heapq
import random

# -------------------------
# The production code (your enhanced ticketing system)
# -------------------------
class Ticket:
    def __init__(self, ticket_id, name, number, email, priority, seat=None):
        self.ticket_id = ticket_id
        self.name = name
        self.number = number
        self.email = email
        self.priority = priority
        self.seat = seat

    def __repr__(self):
        return (f"TicketID: {self.ticket_id} | Name: {self.name} | Number: {self.number} | "
                f"Email: {self.email} | Seat: {self.seat} | Priority: {self.priority}")

class Seating:
    def __init__(self, rows=20, cols=25):
        self.rows = rows
        self.cols = cols
        self.seats = [[None for _ in range(cols)] for _ in range(rows)]
        self.available_seats = {(r, c) for r in range(rows) for c in range(cols)}
        self.purchased_tickets = {}
        self.tickets_by_name = {}
        self.tickets_by_email = {}
        self.tickets_by_phone = {}
        self.queue = []
        self.job_count = 0

    def display_seats(self):
        print("Current Seating Grid:")
        print("   " + "  ".join([f"{i:2d}" for i in range(self.cols)]))
        print("  " + "----" * self.cols)
        for i in range(self.rows):
            row_str = f"{i:2d} | " + "  ".join(["X" if self.seats[i][j] else "O" for j in range(self.cols)])
            print(row_str)
        print()

    def add_ticket_to_indexes(self, ticket):
        name_key = ticket.name.lower()
        email_key = ticket.email.lower()
        phone_key = ticket.number
        self.tickets_by_name.setdefault(name_key, set()).add(ticket.ticket_id)
        self.tickets_by_email.setdefault(email_key, set()).add(ticket.ticket_id)
        self.tickets_by_phone.setdefault(phone_key, set()).add(ticket.ticket_id)

    def remove_ticket_from_indexes(self, ticket):
        name_key = ticket.name.lower()
        email_key = ticket.email.lower()
        phone_key = ticket.number
        if name_key in self.tickets_by_name:
            self.tickets_by_name[name_key].discard(ticket.ticket_id)
        if email_key in self.tickets_by_email:
            self.tickets_by_email[email_key].discard(ticket.ticket_id)
        if phone_key in self.tickets_by_phone:
            self.tickets_by_phone[phone_key].discard(ticket.ticket_id)

    def generate_unique_ticket_id(self):
        # Check both purchased and queued ticket IDs.
        ticket_set = set(self.purchased_tickets.keys())
        ticket_set.update({t[2].ticket_id for t in self.queue})
        while True:
            ticket_id = random.randint(1000, 4000)
            if ticket_id not in ticket_set:
                return ticket_id

    def cancel_ticket(self, ticket_id):
        try:
            if ticket_id in self.purchased_tickets:
                cancelled_ticket = self.purchased_tickets[ticket_id]
                row, col = cancelled_ticket.seat
                self.seats[row][col] = None
                self.available_seats.add((row, col))
                del self.purchased_tickets[ticket_id]
                self.remove_ticket_from_indexes(cancelled_ticket)
                print(f"Ticket {ticket_id} cancelled.")
                # If a waitlisted ticket exists, assign the freed seat.
                if self.queue:
                    priority, order, new_ticket = heapq.heappop(self.queue)
                    new_ticket.seat = (row, col)
                    self.purchased_tickets[new_ticket.ticket_id] = new_ticket
                    self.seats[row][col] = new_ticket
                    self.available_seats.discard((row, col))
                    print(f"Waitlisted ticket {new_ticket.ticket_id} assigned to seat {(row, col)}.")
                return
            # If not in purchased tickets, search in the queue.
            new_queue = []
            found = False
            for item in self.queue:
                if item[2].ticket_id == ticket_id:
                    found = True
                    self.remove_ticket_from_indexes(item[2])
                    continue
                new_queue.append(item)
            if found:
                self.queue = new_queue
                heapq.heapify(self.queue)
                print(f"Waitlisted ticket {ticket_id} removed.")
            else:
                print(f"Ticket {ticket_id} not found.")
        except Exception as e:
            print(f"Error cancelling ticket {ticket_id}: {e}")

    def modifications(self, ticket_id, new_priority=None, new_row=None, new_col=None):
        try:
            if new_row is not None or new_col is not None:
                if new_row < 0 or new_row >= self.rows or new_col < 0 or new_col >= self.cols:
                    return "New seat is out of bounds."
                if (new_row, new_col) not in self.available_seats:
                    return "New seat is already taken."
            if ticket_id in self.purchased_tickets:
                ticket = self.purchased_tickets[ticket_id]
                if new_priority is not None:
                    ticket.priority = new_priority
                    return f"Ticket {ticket_id} priority updated to {new_priority}."
                if new_row is not None and new_col is not None:
                    old_seat = ticket.seat
                    self.seats[old_seat[0]][old_seat[1]] = None
                    self.available_seats.add(old_seat)
                    ticket.seat = (new_row, new_col)
                    self.seats[new_row][new_col] = ticket
                    self.available_seats.discard((new_row, new_col))
                    return f"Ticket {ticket_id} seat changed from {old_seat} to {(new_row, new_col)}."
            # Modify in waitlist if present
            new_queue = []
            found_in_queue = False
            for item in self.queue:
                if item[2].ticket_id == ticket_id:
                    found_in_queue = True
                    item[2].priority = new_priority if new_priority is not None else item[2].priority
                    new_item = (item[2].priority, item[1], item[2])
                    new_queue.append(new_item)
                else:
                    new_queue.append(item)
            if found_in_queue:
                self.queue = new_queue
                heapq.heapify(self.queue)
                return f"Waitlisted ticket {ticket_id} updated with new priority {new_priority}."
            return f"Ticket {ticket_id} not found."
        except Exception as e:
            return f"Error updating ticket {ticket_id}: {e}"

    def lookup(self, ticket_id):
        if ticket_id in self.purchased_tickets:
            return self.purchased_tickets[ticket_id]
        for item in self.queue:
            if item[2].ticket_id == ticket_id:
                return item[2]
        return "Ticket not found."

    def search_by_attribute(self, field, value):
        value = value.lower() if field in ["name", "email"] else value
        matching_ids = set()
        if field == "name":
            matching_ids = self.tickets_by_name.get(value, set())
        elif field == "email":
            matching_ids = self.tickets_by_email.get(value, set())
        elif field == "phone":
            matching_ids = self.tickets_by_phone.get(value, set())
        else:
            return "Invalid search field."
        results = []
        for tid in matching_ids:
            if tid in self.purchased_tickets:
                results.append(self.purchased_tickets[tid])
            else:
                for item in self.queue:
                    if item[2].ticket_id == tid:
                        results.append(item[2])
        return results if results else "No tickets found for given search criteria."

    def get_seat_price_with_priority(self, row, col, priority):
        base_price = 50
        multiplier = 1.0
        if priority == 1:
            multiplier = 1.2
        elif priority == 2:
            multiplier = 1.0
        elif priority == 3:
            multiplier = 0.8
        premium = max(0, (self.rows - row)) * 2
        return (base_price + premium) * multiplier

    def recommend_seat(self, priority):
        # Divide seating into three tiers based on row number.
        if priority == 1:
            tier_range = range(0, max(1, self.rows // 3))
        elif priority == 2:
            tier_range = range(self.rows // 3, 2 * self.rows // 3)
        elif priority == 3:
            tier_range = range(2 * (self.rows // 3), self.rows)
        else:
            tier_range = range(0, self.rows)
        best_seat = None
        best_price = -float("inf")
        for seat in self.available_seats:
            r, c = seat
            if r in tier_range:
                price = self.get_seat_price_with_priority(r, c, priority)
                if price > best_price:
                    best_price = price
                    best_seat = seat
        if best_seat:
            return f"Recommended seat for priority {priority}: {best_seat} with price ${best_price:.2f}."
        else:
            return f"No available seats in tier for priority {priority}."

    def expand_seating(self, new_capacity):
        current_capacity = self.rows * self.cols
        if new_capacity <= current_capacity:
            return f"New capacity must be greater than current capacity ({current_capacity})."
        additional_seats = new_capacity - current_capacity
        extra_rows = additional_seats // self.cols
        old_rows = self.rows
        for _ in range(extra_rows):
            self.seats.append([None for _ in range(self.cols)])
            for col in range(self.cols):
                self.available_seats.add((self.rows, col))
            self.rows += 1
        popped = 0
        for r in range(old_rows, self.rows):
            for c in range(self.cols):
                if self.seats[r][c] is None and self.queue:
                    priority, order, ticket = heapq.heappop(self.queue)
                    ticket.seat = (r, c)
                    self.purchased_tickets[ticket.ticket_id] = ticket
                    self.seats[r][c] = ticket
                    self.available_seats.discard((r, c))
                    popped += 1
                if not self.queue:
                    break
            if not self.queue:
                break
        self.display_seats()
        return (f"Added {extra_rows} extra rows (total new capacity {new_capacity}). "
                f"{popped} waitlisted tickets were assigned seats.")

# -------------------------
# Test Suite: Generating Test Data & Handling Edge Cases
# -------------------------
def run_tests():
    print("=== Test Suite for Enhanced Ticketing System ===\n")

    # --- Test Case 1: Single Ticket Booking (valid input) ---
    print("Test Case 1: Single Ticket Booking")
    test_system = Seating(rows=3, cols=3)  # small grid for testing
    # Simulate booking a ticket by directly creating a Ticket instance.
    ticket1 = Ticket(ticket_id=1010, name="Alice", number="1234567890",
                     email="alice@example.com", priority=1, seat=(0, 0))
    test_system.purchased_tickets[ticket1.ticket_id] = ticket1
    test_system.seats[0][0] = ticket1
    test_system.available_seats.remove((0, 0))
    test_system.add_ticket_to_indexes(ticket1)
    print("Booked Ticket:")
    print(ticket1)
    test_system.display_seats()

    # --- Test Case 2: Modify Ticket with Invalid Seat Coordinates ---
    print("Test Case 2: Modify Ticket to Invalid Seat Coordinates")
    # Attempt to change seat for ticket1 to an out-of-bound location (-1, 5)
    result = test_system.modifications(ticket1.ticket_id, new_row=-1, new_col=5)
    print("Expected error message:", result)

    # --- Test Case 3: Booking a Seat That Is Already Taken ---
    print("\nTest Case 3: Booking a Seat that is Already Taken")
    # Ticket for Bob attempting to use seat (0,0) (which is already booked)
    ticket2 = Ticket(ticket_id=1011, name="Bob", number="0987654321",
                     email="bob@example.com", priority=1, seat=(0, 0))
    if (0, 0) not in test_system.available_seats:
        print("Seat (0,0) is correctly detected as taken. Cannot book seat (0,0).")
    else:
        print("Error: Seat (0,0) should be taken but is available.")

    # --- Test Case 4: Waitlist Booking when No Seats are Available ---
    print("\nTest Case 4: Waitlist Booking Scenario")
    # Fill the grid so no available seats remain.
    for r in range(test_system.rows):
        for c in range(test_system.cols):
            if (r, c) in test_system.available_seats:
                ticket = Ticket(ticket_id=1000 + r*3 + c, name=f"User{r}{c}",
                                number="000", email="user@example.com",
                                priority=1, seat=(r, c))
                test_system.purchased_tickets[ticket.ticket_id] = ticket
                test_system.seats[r][c] = ticket
                test_system.available_seats.remove((r, c))
                test_system.add_ticket_to_indexes(ticket)
    test_system.display_seats()
    # Now simulate adding a ticket to the waitlist.
    ticket_waitlist = Ticket(ticket_id=1050, name="Charlie", number="555",
                             email="charlie@example.com", priority=2, seat=None)
    heapq.heappush(test_system.queue, (ticket_waitlist.priority, test_system.job_count, ticket_waitlist))
    test_system.job_count += 1
    test_system.add_ticket_to_indexes(ticket_waitlist)
    print("Waitlist ticket added:")
    print(ticket_waitlist)

    # --- Test Case 5: Group Booking (Contiguous Block Available) ---
    print("\nTest Case 5: Group Booking with Contiguous Seats")
    # Reinitialize a fresh system for group booking.
    test_group = Seating(rows=3, cols=5)
    # We simulate group booking manually:
    group_details = [
        {'name': 'David', 'number': '111', 'email': 'david@example.com'},
        {'name': 'Eve', 'number': '222', 'email': 'eve@example.com'},
        {'name': 'Faythe', 'number': '333', 'email': 'faythe@example.com'}
    ]
    row, start_col = 0, 1  # assume row 0 has contiguous free seats starting at column 1
    assigned_tickets = []
    for i, details in enumerate(group_details):
        tid = 2000 + i
        seat = (row, start_col + i)
        ticket = Ticket(ticket_id=tid, name=details['name'], number=details['number'],
                        email=details['email'], priority=1, seat=seat)
        assigned_tickets.append(ticket)
        test_group.purchased_tickets[tid] = ticket
        test_group.seats[row][start_col + i] = ticket
        test_group.available_seats.remove(seat)
        test_group.add_ticket_to_indexes(ticket)
    test_group.display_seats()
    print("Group Tickets:")
    for t in assigned_tickets:
        print(t)

    # --- Test Case 6: Seat Recommendation for Various Priorities ---
    print("\nTest Case 6: Seat Recommendation by Priority")
    test_rec = Seating(rows=6, cols=6)
    # Initially, the grid is completely empty.
    for p in [1, 2, 3]:
        rec = test_rec.recommend_seat(p)
        print(f"Recommendation for priority {p}: {rec}")

    # --- Test Case 7: Modify Ticket to an Occupied Seat ---
    print("\nTest Case 7: Modify Ticket to an Already Occupied Seat")
    # Book a ticket for Frank
    ticket_frank = Ticket(ticket_id=3000, name="Frank", number="333",
                          email="frank@example.com", priority=2, seat=(1,1))
    test_rec.purchased_tickets[ticket_frank.ticket_id] = ticket_frank
    test_rec.seats[1][1] = ticket_frank
    if (1,1) in test_rec.available_seats:
        test_rec.available_seats.remove((1,1))
    test_rec.add_ticket_to_indexes(ticket_frank)
    # Attempt to move Frank to seat (0,0)
    # Since (0,0) is still available (unless taken in a previous test), it should work.
    result = test_rec.modifications(ticket_frank.ticket_id, new_row=0, new_col=0)
    print("Modification result (if seat (0,0) is available):", result)
    # Now, if we force a conflict by booking (0,1) and then trying to move Frank there:
    conflict_ticket = Ticket(ticket_id=3001, name="Grace", number="444",
                             email="grace@example.com", priority=1, seat=(0,1))
    test_rec.purchased_tickets[conflict_ticket.ticket_id] = conflict_ticket
    test_rec.seats[0][1] = conflict_ticket
    if (0,1) in test_rec.available_seats:
        test_rec.available_seats.remove((0,1))
    test_rec.add_ticket_to_indexes(conflict_ticket)
    result_conflict = test_rec.modifications(ticket_frank.ticket_id, new_row=0, new_col=1)
    print("Modification result when moving to an occupied seat (0,1):", result_conflict)

    # --- Test Case 8: Search by Attribute ---
    print("\nTest Case 8: Search for Tickets by Attribute")
    # Search for ticket using name "David" from the group booking.
    results = test_group.search_by_attribute("name", "david")
    print("Search for 'David':", results)

    # --- Test Case 9: Expand Seating Capacity (Edge Case: new_capacity <= current capacity) ---
    print("\nTest Case 9: Expand Seating Capacity with Invalid New Capacity")
    # For a 6x6 grid, current capacity is 36.
    expand_result_invalid = test_rec.expand_seating(36)
    print(expand_result_invalid)

    # --- Test Case 10: Expand Seating Capacity and Assign Waitlist ---
    print("\nTest Case 10: Expand Seating Capacity and Assign Waitlist Tickets")
    # Add a waitlist member to test_rec.
    ticket_waitlist2 = Ticket(ticket_id=4000, name="Heidi", number="777",
                              email="heidi@example.com", priority=1, seat=None)
    heapq.heappush(test_rec.queue, (ticket_waitlist2.priority, test_rec.job_count, ticket_waitlist2))
    test_rec.job_count += 1
    test_rec.add_ticket_to_indexes(ticket_waitlist2)
    expand_result = test_rec.expand_seating(50)  # Expanding to a larger capacity.
    print(expand_result)

if __name__ == "__main__":
    run_tests()


=== Test Suite for Enhanced Ticketing System ===

Test Case 1: Single Ticket Booking
Booked Ticket:
TicketID: 1010 | Name: Alice | Number: 1234567890 | Email: alice@example.com | Seat: (0, 0) | Priority: 1
Current Seating Grid:
    0   1   2
  ------------
 0 | X  O  O
 1 | O  O  O
 2 | O  O  O

Test Case 2: Modify Ticket to Invalid Seat Coordinates
Expected error message: New seat is out of bounds.

Test Case 3: Booking a Seat that is Already Taken
Seat (0,0) is correctly detected as taken. Cannot book seat (0,0).

Test Case 4: Waitlist Booking Scenario
Current Seating Grid:
    0   1   2
  ------------
 0 | X  X  X
 1 | X  X  X
 2 | X  X  X

Waitlist ticket added:
TicketID: 1050 | Name: Charlie | Number: 555 | Email: charlie@example.com | Seat: None | Priority: 2

Test Case 5: Group Booking with Contiguous Seats
Current Seating Grid:
    0   1   2   3   4
  --------------------
 0 | O  X  X  X  O
 1 | O  O  O  O  O
 2 | O  O  O  O  O

Group Tickets:
TicketID: 2000 | Name: David | Num